# Análise finaceira
Análise finaceira do comportamento do modelo em diferentes janelas de tempo e diferentes estratégias

### Pacotes

In [1]:
using LinearAlgebra;
using CSV;
using DataFrames;
using JuMP;
using SpecialFunctions;
using Ipopt;
using Plots;
using StatsBase;

### Dados

In [2]:
#retornos
tabelaRetorno=CSV.File("C://Users//Rafaela//Desktop//Documentos Gerais//Tese-Finanças//dados//retornos40_af.csv");
DataRetornos = DataFrame(tabelaRetorno);

#indices
M=39 #quantidade de ativos
L=3 

χ=[1/3 1/3 1/3]#Perfil de risco

α=[0.6 , 0.75,  0.9 ] #nível de confiança

3-element Vector{Float64}:
 0.6
 0.75
 0.9

### Janelas de tempo

In [3]:
baseAno=groupby(DataRetornos,:ano)#agrupando por ano
baseMes=groupby(DataRetornos,:periodo)#agrupando por mes

#Bases anuais
# Ano2007=baseAno[1]; #Ano 2007
# Ano2008=baseAno[2]; #Ano 2008
# Ano2009=baseAno[3]; #Ano 2009
# Ano2010=baseAno[4]; #Ano 2010
# Ano2011=baseAno[5]; #Ano 2011
# Ano2012=baseAno[6]; #Ano 2012
# Ano2013=baseAno[7]; #Ano 2013
# Ano2014=baseAno[8]; #Ano 2014
# Ano2015=baseAno[9]; #Ano 2015
# Ano2016=baseAno[10]; #Ano 2016
# Ano2017=baseAno[11]; #Ano 2017
# Ano2018=baseAno[12]; #Ano 2018
# Ano2019=baseAno[13]; #Ano 2019
# Ano2020=baseAno[14]; #Ano 2020
# Ano2021=baseAno[15];#Ano 2021

#bases trienais
# Trimestre0709=vcat(Ano2007,Ano2008,Ano2009);
# Trimestre1012=vcat(Ano2010,Ano2011,Ano2012);
# Trimestre1315=vcat(Ano2013,Ano2014,Ano2015);
# Trimestre1618=vcat(Ano2016,Ano2017,Ano2018);
# Trimestre1921=vcat(Ano2019,Ano2020,Ano2021);


### Modelagem Versão 1 (sem restrição de retorno)

In [4]:
function MOD_V1(periodo)   
   dados=periodo
   N=size(dados,1)
   retornos=zeros(N,M)
   retornos.=dados[1:N,5:(M+4)]
   #----------------------------
   δ=0.0

    modelT1 = Model(Ipopt.Optimizer) #Modelo 1-versão 1

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    #@constraint(modelT1, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    #função substituta f1--------------------------------
    η=2^(-13)        #parâmetro da função f1, menor melhor
   
    f1(a) = max(a,0)    

    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)       # a função f1
    d2f1(a)= η/(2*(η+a^2)^(3/2))    # a derivada da função f1
    
    #-------------------------------------------------------
    
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
   
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 

 
    optimize!(modelT1) #otimizar
   
        #solução
        pesoT1= value.(x); 
        #VaR
        VaRT1= value.(y); 
        uT1=value.(u);
        #CVaR misto
        CVaRmistoT1=round(objective_value(modelT1);digits=3);
         
        #retorno acumulado
        retorno1= sum(transpose(pesoT1)*retornos[t,:] for t=1:N);

        #retorno real
        #capital=100
        #retornoDiarioCarteiraOtima= retornos*pesoT1;
        #retornoReal=zeros(N);
        #for i in 1:N
        # retornoReal[i]=capital+capital*retornoDiarioCarteiraOtima[i]
        # capital=retornoReal[i]
        #end
         #RR=(retornoReal[N]-100)/100;

         #máximo drawdown
         maxD=maximum(uT1);

         #Drawdown médio
         mediaD=mean(uT1);
    
        return CVaRmistoT1 , retorno1
end

MOD_V1 (generic function with 1 method)

### Risco e retorno da carteira quadrimestral

In [5]:
retorno2=zeros(45)

for i in 1:10
  retorno2[i]=MOD_V1(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
end 


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     2764
Number of nonzeros in Lagrangian Hessian.............:      327

Total number of variables............................:      123
                     variables with only lower bounds:       81
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equal

In [6]:
for i in 11:20
  retorno2[i]=MOD_V1(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3078
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [7]:
for i in 21:30
  retorno2[i]=MOD_V1(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3119
Number of nonzeros in Lagrangian Hessian.............:      331

Total number of variables............................:      124
                     variables with only lower bounds:       82
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       81
        inequality constraints with only lower bounds:       81
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [8]:
for i in 31:40
  retorno2[i]=MOD_V1(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3094
Number of nonzeros in Lagrangian Hessian.............:      323

Total number of variables............................:      122
                     variables with only lower bounds:       80
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       79
        inequality constraints with only lower bounds:       79
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [9]:
for i in 41:45
  retorno2[i]=MOD_V1(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3417
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [14]:
retorno2

45-element Vector{Float64}:
  4.786993572118778e-5
  7.471506516036795e-6
  5.831878904304935e-6
  3.200679364048534e-6
 -0.011179346523438407
 -9.886905762734166e-8
  7.90071282976215e-6
  1.7616634854754076e-5
  2.1741062403910877e-5
  1.2999772404512013
  6.188478423563336e-6
  1.1302348966726861e-5
  0.29911559043704294
  ⋮
  0.15160581543611595
  1.110370851166445e-6
  0.17850520555517127
  0.15824453609955288
  0.08334816403197014
  0.29417529825909683
  0.01839938519525225
  0.44629376582994484
  0.3679722296434415
  0.3600169146864547
  0.09975228478262141
  0.06839093076419873

### Fronteira eficiente

### Modelo versão 1( com restrição de retorno)

In [167]:
function MOD_V1_R(periodo, δ)   
   dados=periodo
   N=size(dados,1)
   retornos=zeros(N,M)
   retornos.=dados[1:N,5:(M+4)]
   #----------------------------
 
    modelT1 = Model(Ipopt.Optimizer) #Modelo 1-versão 1

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    @constraint(modelT1, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    #função substituta f1--------------------------------
    η=2^(-13)        #parâmetro da função f1, menor melhor
   
    f1(a) = max(a,0)    

    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)       # a função f1
    d2f1(a)= η/(2*(η+a^2)^(3/2))    # a derivada da função f1
    
    #-------------------------------------------------------
    
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
   
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 
 
    optimize!(modelT1) #otimizar
   
        #solução
        pesoT1= value.(x); 
        #VaR
        VaRT1= value.(y); 
        uT1=value.(u);
        #CVaR misto
        CVaRmistoT1=round(objective_value(modelT1);digits=5);
        #retorno acumulado
        retorno1= sum(transpose(pesoT1)*retornos[t,:] for t=1:N);
    
        return CVaRmistoT1 , retorno1
end

MOD_V1_R (generic function with 1 method)

### Fronteira eficiente em período de crise

In [158]:
Quarter2_2008=vcat(baseMes[17],baseMes[18],baseMes[19],baseMes[20]) #Segundo quadrimestre de 2008
δ3=[0.04, 0.08, 0.12, 0.16, 0.2,0.22, 0.24]
retorno3=zeros(7)
risco3=zeros(7)

for i in 1:7
Aux=MOD_V1_R(Quarter2_2008, δ3[i])
retorno3[i]=Aux[2]
risco3[i]=Aux[1]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3018
Number of nonzeros in Lagrangian Hessian.............:      339

Total number of variables............................:      126
                     variables with only lower bounds:       84
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [160]:
risco3
#retorno3

7-element Vector{Float64}:
 0.03999999918822264
 0.07999999281462251
 0.11999999143554446
 0.15999998920321468
 0.19999997611335482
 0.21999996266192073
 0.23999995013774988

### Fronteira eficiente em período de NÃO crise

In [182]:
Quarter2_2013=vcat(baseMes[80],baseMes[81],baseMes[82],baseMes[83]) #Segundo quadrimestre de 2013
δ4=[0.2, 0.32, 0.36, 0.4, 0.44, 0.48, 0.52]
retorno4=zeros(7)
risco4=zeros(7)

for i in 1:7
Aux4=MOD_V1_R(Quarter2_2013, δ4[i])
retorno4[i]=Aux4[2]
risco4[i]=Aux4[1]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3266
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       85
        inequality constraints with only lower bounds:       85
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [183]:
risco4
#retorno4

7-element Vector{Float64}:
 0.01147
 0.01471
 0.01592
 0.01722
 0.01877
 0.02091
 0.02409

### Modelagem- Modelo original

In [132]:
function MOD0(periodo, δ )
   dados=periodo
   N=size(dados,1)
   retornos=zeros(N,M)
   retornos.=dados[1:N,5:(M+4)]
   #----------------------------
       modelT1 = Model(Ipopt.Optimizer)

       @variables(modelT1, begin
       1 ≥ x[1:M] ≥ 0
       u[1:N] ≥ 0 
       y[1:L] 
       z[1:N,1:L] ≥ 0
        end)
   
      @constraint(modelT1,sum(x[i] for i=1:M)== 1);
      @constraint(modelT1, u[1]==0 );
      @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
      @constraint(modelT1, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
      @constraint(modelT1,[t=1:N,j=1:L], u[t]-y[j] ≤ z[t,j]);
  
      @objective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(z[t,j] for t=1:N )) for j=1:L));
   
      optimize!(modelT1); #otimizar

        #solução
        pesoT1= value.(x); 
        #VaR
        VaRT1= value.(y); 
        zT1= value.(z);
        uT1=value.(u);
        #CVaR misto
        CVaRmistoT1=round(objective_value(modelT1);digits=3);
         
         #retorno acumulado
         retorno1= sum(transpose(pesoT1)*retornos[t,:] for t=1:N);

         #retorno real
         capital=100
         retornoDiarioCarteiraOtima= retornos*pesoT1;
         retornoReal=zeros(N);
         for i in 1:N
          retornoReal[i]=capital+capital*retornoDiarioCarteiraOtima[i]
          capital=retornoReal[i]
         end
         RR=(retornoReal[N]-100)/100;

         #máximo drawdown
         maxD=maximum(uT1);

         #Drawdown médio
         mediaD=mean(uT1);
    
        return CVaRmistoT1 , retorno1
end

MOD0 (generic function with 1 method)

**EXEMPLOS**

In [6]:
#MODM(baseMes[1]); #Exemplo Jan/2007 -alterar a janela de tempo
MODM(vcat(baseMes[1], baseMes[2],baseMes[3],baseMes[4])) #Exemplo 1º quadrimestre de 2007 -alterar a janela de tempo


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     2764
Number of nonzeros in Lagrangian Hessian.............:      327

Total number of variables............................:      123
                     variables with only lower bounds:       81
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equal

(0.008, 4.787095132613217e-5)

### CVaR misto e Retorno real quadrimestral

In [5]:
CVaRmisto4meses=zeros(45)
RR4meses=zeros(45)

for i in 1:5
   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
end 

# for i in 6:10
#   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end 

# for i in 11:15
#   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end 

# for i in 16:20
#   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end

# for i in 21:25
#   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end

# for i in 26:30
#  CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end

#for i in 31:35
 #  CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
 #   RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end


# for i in 36:40
#   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end

# for i in 41:45
#   CVaRmisto4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[1]
#    RR4meses[i]=MODM(vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]))[2]
# end
    


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     2764
Number of nonzeros in Lagrangian Hessian.............:      327

Total number of variables............................:      123
                     variables with only lower bounds:       81
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equal

### Resultado 1

In [25]:
resultadosA=DataFrame(cvar=CVaRmisto4meses, retorno=  RR4meses)

"C://Users//Rafaela//Documents//Tese_doutorado//Output//RetornoRiscoA.csv"

In [26]:
CSV.write("C://Users//Rafaela//Documents//Tese_doutorado//Output//RetornoRiscoA.csv",resultadosA)

,cvar,retorno
,Float64,Float64
1,0.008,4.7871e-5
2,0.008,7.47153e-6
3,0.008,5.83189e-6
4,0.008,3.20068e-6
5,0.019,-0.0117106
6,0.008,-9.88691e-8
7,0.008,7.90074e-6
8,0.008,1.76168e-5
9,0.008,2.17413e-5


## Rebalanceamento

**Modelagem**

In [5]:
function MODMR(periodo)   
   dados=periodo
   N=size(dados,1)
   retornos=zeros(N,M)
   retornos.=dados[1:N,5:(M+4)]
   #----------------------------
   δ=0.0

    modelT1 = Model(Ipopt.Optimizer) #Modelo 1-versão 1

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0   #força a entrada de todos os ativos na carteira
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*retornos[t,:]);
    #@constraint(modelT1, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    #função substituta f1--------------------------------
    η=2^(-13)        #parâmetro da função f1, menor melhor
   
    f1(a) = max(a,0)    

    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)       # a função f1
    d2f1(a)= η/(2*(η+a^2)^(3/2))    # a derivada da função f1
    
    #-------------------------------------------------------
    
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
   
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 

 
    optimize!(modelT1) #otimizar
   
        #solução
        pesoT1= value.(x); 
        #VaR
        VaRT1= value.(y); 
        uT1=value.(u);
        #CVaR misto
        CVaRmistoT1=round(objective_value(modelT1);digits=3);
         
        #retorno acumulado
        retorno1= sum(transpose(pesoT1)*retornos[t,:] for t=1:N);

        #retorno real
        #capital=100
        #retornoDiarioCarteiraOtima= retornos*pesoT1;
        #retornoReal=zeros(N);
        #for i in 1:N
         #retornoReal[i]=capital+capital*retornoDiarioCarteiraOtima[i]
         #capital=retornoReal[i]
        #end
         #RR=(retornoReal[N]-100)/100;

         #máximo drawdown
        # maxD=maximum(uT1);

         #Drawdown médio
         #mediaD=mean(uT1);
    
        return CVaRmistoT1 , retorno1 , pesoT1
end



MODMR (generic function with 1 method)

**Algoritmo**

In [5]:
LimiteDrawdown=0.1
drawdown=zeros(45)
RR1=zeros(45)
pesos =ones(39)*1/39

for i in 2:10
    retornosAux=vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]);
    N=size(retornosAux,1)
    retornosAux2=zeros(N,M)
    retornosAux2.=retornosAux[1:N,5:(M+4)]
    
   if drawdown[i-1] < LimiteDrawdown
       rAcum=zeros(N);
       for j in 1:N
         rAcum[j]= sum(transpose(pesos)*retornosAux2[t,:] for t=1:j)
       end
       d1=zeros(N);
       for k in 1:N
         d1[k]= maximum(rAcum[1:k])-rAcum[k]
       end
       drawdown[i]=maximum(d1)
   else
     RR1[i], drawdown[i], pesos= MODMR(retornosAux)[2], MODMR(retornosAux)[3], MODMR(retornosAux)[4]
    
   end
end
    


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     2791
Number of nonzeros in Lagrangian Hessian.............:      315

Total number of variables............................:      120
                     variables with only lower bounds:       78
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equal

LoadError: BoundsError: attempt to access Tuple{Float64, Float64, Vector{Float64}} at index [4]

In [10]:
     pesos

39-element Vector{Float64}:
 2.9976555119869918e-6
 5.929232370765329e-8
 4.312018182810478e-7
 2.350707888299198e-7
 1.8793112463592972e-7
 2.422639454427582e-7
 8.199167011027739e-7
 2.2643585628764938e-7
 3.0833994226448474e-7
 5.636087922433319e-7
 2.047748894183474e-7
 7.667869817646564e-6
 1.1722523793887934e-7
 ⋮
 4.4008650271487e-7
 1.3942252441163195e-6
 6.00988089226402e-7
 2.6653256479148847e-7
 8.825579482407547e-6
 8.798447700736806e-7
 2.6804961822999847e-6
 0.4999797107751085
 7.826956960348859e-8
 1.3494188013494215e-7
 1.9477952876239166e-6
 1.8679125996223177e-7

In [14]:
LimiteDrawdown=0.1
#drawdown=zeros(45)
#risco1=zeros(45)
#retorno1=zeros(45)
#pesos =ones(39)*1/39

for i in 31:45
    retornosAux=vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]);
    N=size(retornosAux,1)
    retornosAux2=zeros(N,M)
    retornosAux2.=retornosAux[1:N,5:(M+4)]
    
   if drawdown[i-1] < LimiteDrawdown
       rAcum=zeros(N);
       for j in 1:N
         rAcum[j]= sum(transpose(pesos)*retornosAux2[t,:] for t=1:j)
       end
       d1=zeros(N);
       for k in 1:N
         d1[k]= maximum(rAcum[1:k])-rAcum[k]
       end
       retorno1[i]=rAcum[N] 
       drawdown[i]=maximum(d1)
   else
    risco1[i], retorno1[i]= MODMR(retornosAux)[1], MODMR(retornosAux)[2]
    
   end
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3094
Number of nonzeros in Lagrangian Hessian.............:      323

Total number of variables............................:      122
                     variables with only lower bounds:       80
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       79
        inequality constraints with only lower bounds:       79
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [15]:
retorno1[31:45]

15-element Vector{Float64}:
  0.07044485130479748
  0.1393852303076923
  5.448262956498209e-6
  0.10783756169230767
 -0.12542921571794874
  0.17850520555517127
  0.12460806871794876
  0.1034291431794872
  0.19273809328205121
 -0.24551946415384615
  0.44629376582994484
  0.17906636041025645
  0.10381631228205127
 -0.005632968615384586
  0.06839093076419873

### Pesos iguais

In [9]:
LimiteDrawdown=0.1
#drawdown=zeros(45)
#risco1=zeros(45)
#retorno1=zeros(45)
#pesos =ones(39)*1/39

for i in 16:45
    retornosAux=vcat(baseMes[4*i-3],baseMes[4*i-2],baseMes[4*i-1],baseMes[4*i]);
    N=size(retornosAux,1)
    retornosAux2=zeros(N,M)
    retornosAux2.=retornosAux[1:N,5:(M+4)]
    
   #if drawdown[i-1] < LimiteDrawdown
       rAcum=zeros(N);
       for j in 1:N
         rAcum[j]= sum(transpose(pesos)*retornosAux2[t,:] for t=1:j)
       end
       d1=zeros(N);
       for k in 1:N
         d1[k]= maximum(rAcum[1:k])-rAcum[k]
       end
       retorno1[i]=rAcum[N] 
       drawdown[i]=maximum(d1)
   #else
    #risco1[i], retorno1[i]= MODMR(retornosAux)[1], MODMR(retornosAux)[2]
    
  # end
end

In [14]:
drawdown[31:45]
sum(retorno1[i] for i=1:45)

2.022058644974359